## Evaluation notebook

### 1. Install / Import

In [1]:
import json
from pdf2image import convert_from_path
from common_utils import embed_texts, embed_images, encode_image_to_base64, search_index, retrieve_context, call_gpt_4, extract_figures_from_pdf
from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ipynb.fs.full.classRag import RAG
from dotenv import load_dotenv
from PIL import Image
load_dotenv()
from PyPDF2 import PdfReader

c:\Skola\Master's Thesis\masters-thesis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. EXTRACT IMAGES

In [2]:
file ="../knowledge/subset_monetary_policy_report.pdf"

import cv2
import numpy as np
from PIL import Image
import os
import fitz  # PyMuPDF

def extract_images_from_pdf(pdf_path, output_folder="extracted_data", padding=300, xpadding = 300):
    os.makedirs(output_folder, exist_ok=True)  # Create output folder
    image_paths = []
    
    # Open the PDF document
    doc = fitz.open(pdf_path)
    
    # Loop through all pages in the PDF
    for page_index in range(len(doc)):
        page = doc.load_page(page_index)  # Load the page
        
        # Rasterize the page to an image
        pix = page.get_pixmap(dpi=300)  # Convert to image with high DPI
        full_image_path = os.path.join(output_folder, f"full_page_{page_index + 1}.png")
        pix.save(full_image_path)
        
        # Convert the image to OpenCV format (numpy array)
        full_image = cv2.imread(full_image_path)
        
        # Convert to grayscale
        gray_image = cv2.cvtColor(full_image, cv2.COLOR_BGR2GRAY)
        
        # Threshold to create a binary image (to highlight potential image areas)
        _, thresh = cv2.threshold(gray_image, 240, 255, cv2.THRESH_BINARY_INV)
        
        # Find contours (regions that are "boxes" in the image)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Loop through contours and crop the image regions
        img_index = 0
        for contour in contours:
            # Get the bounding box of each contour (x, y, width, height)
            x, y, w, h = cv2.boundingRect(contour)
            
            # Ignore small areas (you can adjust the threshold for min area size)
            if w > 50 and h > 50:
                # Add padding to the bounding box
                x_padded = max(x - padding, 0)  # Ensure x doesn't go below 0
                y_padded = max(y - padding, 0)  # Ensure y doesn't go below 0
                w_padded = min(w + 2 * xpadding, full_image.shape[1] - x_padded)  # Ensure width doesn't exceed image
                h_padded = min(h + 2 * padding, full_image.shape[0] - y_padded)  # Ensure height doesn't exceed image
                
                # Crop the image with padding
                cropped_image = full_image[y_padded:y_padded + h_padded, x_padded:x_padded + w_padded]
                
                # Convert cropped image to PIL format to save it as PNG
                pil_image = Image.fromarray(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                img_filename = f"page_{page_index + 1}_image_{img_index + 1}.png"
                img_path = os.path.join(output_folder, img_filename)
                pil_image.save(img_path, "PNG")
                
                image_paths.append({"image_path": img_path})
                img_index += 1
    
    doc.close()
    return image_paths

# image_paths = extract_images_from_pdf(file)


### 3. EXTRACT TEXT

In [3]:
def extract_text_from_pdf(pdf_path):
    text_data = []
    # Extract text
    reader = PdfReader(file)
    num_pages = len(reader.pages)

    for page_i in range(num_pages):
        page = reader.pages[page_i]
        page_text = page.extract_text()
        
        if page_text and page_text.strip():
            text_data.append({
                "text": page_text.strip(),
                "page_number": page_i + 1
            })
    return text_data

#text_data = extract_text_from_pdf(file)
#print(text_data)



In [4]:
def generate_qa_for_pdf(pdf_path):
    output_json = "QA_" + os.path.basename(pdf_path).replace('.pdf', '.json')

    # Extract images and text from PDF
    image_data = extract_images_from_pdf(pdf_path)
    text_data = extract_text_from_pdf(pdf_path)

    sample_queries = []
    expected_responses = []
    qa_data = []

    # Iterate over text data to generate questions
    for text_info in text_data:
        page_number = text_info["page_number"]
        page_text = text_info["text"]

        user_prompt = json.dumps([
            {"type": "text", "text": (
                "Your task is to formulate a question from the given context while following these rules:\n"
                "1. The question must be answerable using the provided context.\n"
                "2. It should be based on non-trivial information.\n"
                "3. The answer must not contain any links.\n"
                "4. The question should be of moderate difficulty.\n"
                "5. Avoid phrases like 'provided context'.\n"
                "6. The response must be in valid JSON format as follows:\n"
                "{'question': 'Generated question here', 'answer': 'Generated answer here'}"
            )},
            {"type": "text", "text": page_text}
        ])
        
        response_text = call_gpt_4(user_prompt)
        #print(f"Raw API response for text (page {page_number}):", response_text)

        try:
            cleaned_response_text = response_text.strip("```json").strip("```")  # Remove surrounding backticks
            response_data = json.loads(cleaned_response_text)
            question = response_data.get("question")
            answer = response_data.get("answer")

            if question and answer:
                sample_queries.append(question)
                expected_responses.append(answer)
                qa_data.append({"page_number": page_number, "question": question, "answer": answer})
            else:
                print(f"Warning: Missing Q&A data for text on page {page_number}")

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON response for text on page {page_number}: {e}")
            print("Response text:", response_text)
            continue

    # Iterate over image data to generate questions
    for image_info in image_data:
        page_number = image_info["image_path"].split("_")[2]  # Extract page number from image path
        image_path = image_info["image_path"]

        # Convert the image to base64 for processing
        image = Image.open(image_path)
        base64_str = encode_image_to_base64(image)

        user_prompt = json.dumps([
            {"type": "text", "text": (
                "Your task is to formulate a question from the given image while following these rules:\n"
                "1. The question must be answerable using the provided image.\n"
                "2. It should be based on non-trivial information.\n"
                "3. The answer must not contain any links.\n"
                "4. The question should be of moderate difficulty.\n"
                "5. Avoid phrases like 'provided image'.\n"
                "6. The response must be in valid JSON format as follows:\n"
                "{'question': 'Generated question here', 'answer': 'Generated answer here'}"
            )},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_str}"}}
        ])

        response_text = call_gpt_4(user_prompt)
        #print(f"Raw API response for image (page {page_number}):", response_text)

        try:
            cleaned_response_text = response_text.strip("```json").strip("```")  # Remove surrounding backticks
            response_data = json.loads(cleaned_response_text)
            question = response_data.get("question")
            answer = response_data.get("answer")

            if question and answer:
                sample_queries.append(question)
                expected_responses.append(answer)
                qa_data.append({"page_number": page_number, "question": question, "answer": answer})
            else:
                print(f"Warning: Missing Q&A data for image on page {page_number}")

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON response for image on page {page_number}: {e}")
            print("Response text:", response_text)
            continue

    # Output structured data
    output = {
        "sample_queries": sample_queries,
        "expected_responses": expected_responses,
        "qa_data": qa_data
    }

    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(output, f, ensure_ascii=False, indent=4)

    #print("Final Q&A data saved to", output_json)
    return sample_queries, expected_responses, qa_data

In [5]:
if __name__ == "__main__":
    PDF_FILE = "../knowledge/subset_monetary_policy_report.pdf"
    sample_queries, expected_responses, qa_data = generate_qa_for_pdf(pdf_path=PDF_FILE)
    print(sample_queries)
    print(expected_responses)

['What was the total amount of interest-bearing debt for Swedish non-financial companies at the end of 2023, and how was this debt distributed between loans and issued debt securities?', 'What are the main sources of borrowing for large enterprises and SMEs in Sweden as of December 31, 2023, and how does this differ by enterprise size?', 'Which sectors have the highest share of securities borrowing, and what are the primary sources of loans for non-financial companies in Sweden?', 'What were some of the reasons for the increased demand for debt securities issued by Swedish companies after the financial crisis?', "What was the market value of companies' outstanding equities in Sweden at the end of 2023, and how does it compare to the country's GDP?", "What is the significance of the 'q2' symbol in the provided image?", 'What is the primary color scheme used in the image and how does it affect the overall vibe?', 'What is the unique feature of the design on the object shown in the image?